# The NN-Model was trained on Google Colab

In [ ]:
%matplotlib inline
%load_ext tensorboard

import numpy as np
import matplotlib.pyplot as plt
import os

import keras as K
from keras.layers import LeakyReLU
from keras.models import Model
from keras.models import Sequential

from scipy.signal import savgol_filter
from scipy.signal import find_peaks_cwt
from scipy import interpolate

import datetime

import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)


In [ ]:
X=np.load('Xall.npy')
Y=np.load('Yall.npy')
print(X.shape)

nc = 29  # 36 classes
ny = 180 # row nr of xrd-pattern
y_all = np.zeros((nc,ny))
xnew = np.arange(0,90,0.5) 

## During testing several NN-architectures, simple dense networks performed better than 1D convolutional nets

In [ ]:
model = Sequential() 

#init=K.initializers.GlorotUniform(seed=12)
init=K.initializers.HeUniform(seed=123)

wreg1 = K.regularizers.l2(l=2e-3) 
wreg2 = K.regularizers.l2(l=0)

model.add(K.layers.Dense(512,activation='relu',
          kernel_initializer=init,name='d1', input_shape=(180,)))
model.add(K.layers.Dropout(0.1))

model.add(K.layers.Dense(256,activation='relu',name='d2'))
model.add(K.layers.Dropout(0.1))

model.add(K.layers.Dense(256,activation='relu',kernel_regularizer=wreg2,name='d3'))
model.add(K.layers.Dropout(0.1))

model.add(K.layers.Dense(128,kernel_regularizer=wreg2,name='d4'))
model.add(LeakyReLU(alpha=0.1))

model.add(K.layers.Dense(96,kernel_regularizer=wreg2,name='d5'))
model.add(LeakyReLU(alpha=0.1))

model.add(K.layers.Dense(96,kernel_regularizer=wreg2,name='d6'))
model.add(LeakyReLU(alpha=0.1))

model.add(K.layers.Dense(48,kernel_regularizer=wreg2,name='d7'))
model.add(LeakyReLU(alpha=0.1))

model.add(K.layers.Dense(nc, activation='sigmoid',name='d8',kernel_regularizer=wreg1))

optimizer = K.optimizers.Adam(lr=0.6e-4)

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy','binary_crossentropy'], 
              )

model.summary()

In [ ]:
%tensorboard --logdir logs/fit #monitor progess with tensorboard

In [ ]:
BATCH_SIZE=128
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = K.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
chkpt_callback=tf.keras.callbacks.ModelCheckpoint('./model.hdf5', monitor='val_accuracy', 
                                 verbose=1, save_best_only=True, mode='max')

model.fit(X,Y,batch_size=BATCH_SIZE, 
          epochs=100, verbose=1,validation_split=0.1, 
          shuffle=True, 
          callbacks=[tensorboard_callback,chkpt_callback])

### exptected validation accuracy approx. 60% (it's better than it sounds)

## Randomly testing some predictions on whole (train+val) sample set
### (It would be better to create some 'fresh' test-data for this, as hyperparameters might be biased toward the validation set)

In [ ]:
foo=np.random.randint(1,len(Y))
print(foo)
Xtest=X[foo]
plt.plot(Xtest)
print('ACTUAL:')
print(Y[foo])
print(np.argwhere(Y[foo]==1))

foo = np.expand_dims(Xtest, axis=-1).T
print('PREDICTION:')
bar=((model.predict(foo) > 0.5).astype("int32")).squeeze()
print(bar)
print(np.argwhere(bar==1))

## todo: 
- try RNNs
- use generator during fitting instead of precomputed data to increase training set (especially ternary systems are under-represented)